## Data preprocessing

In [20]:
import pandas as pd
import numpy as np

In [ ]:
# Read the CSV file and create a DataFrame
df = pd.read_csv('data/sales_train_evaluation.csv')

# Extraire uniquement les colonnes de ventes journalières (en supposant qu'elles commencent à 'd_1')
daily_sales = df.iloc[:, 6:]

# Créer un dictionnaire de DataFrames, un par produit
product_dfs = {product: data.drop(columns=['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'])
               for product, data in df.groupby('item_id')}

product_dfs['FOODS_1_001_CA_1_validation'].head()

In [14]:
df

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1932,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,4,0,0,0,0,3,3,0,1
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,1,2,1,1,0,0,0,0,0
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,0,2,0,0,0,2,3,0,1
3,HOBBIES_1_004_CA_1_evaluation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,1,0,4,0,1,3,0,2,6
4,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,2,1,0,0,2,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30485,FOODS_3_823_WI_3_evaluation,FOODS_3_823,FOODS_3,FOODS,WI_3,WI,0,0,2,2,...,1,0,3,0,1,1,0,0,1,1
30486,FOODS_3_824_WI_3_evaluation,FOODS_3_824,FOODS_3,FOODS,WI_3,WI,0,0,0,0,...,0,0,0,0,0,0,1,0,1,0
30487,FOODS_3_825_WI_3_evaluation,FOODS_3_825,FOODS_3,FOODS,WI_3,WI,0,6,0,2,...,0,0,1,2,0,1,0,1,0,2
30488,FOODS_3_826_WI_3_evaluation,FOODS_3_826,FOODS_3,FOODS,WI_3,WI,0,0,0,0,...,1,1,1,4,6,0,1,1,1,0


In [15]:
daily_sales

,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,d_10,...,d_1932,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941
0,0,0,0,0,0,0,0,0,0,0,...,2,4,0,0,0,0,3,3,0,1
1,0,0,0,0,0,0,0,0,0,0,...,0,1,2,1,1,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,1,0,2,0,0,0,2,3,0,1
3,0,0,0,0,0,0,0,0,0,0,...,1,1,0,4,0,1,3,0,2,6
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,2,1,0,0,2,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30485,0,0,2,2,0,3,1,4,1,0,...,1,0,3,0,1,1,0,0,1,1
30486,0,0,0,0,0,5,0,1,1,3,...,0,0,0,0,0,0,1,0,1,0
30487,0,6,0,2,2,4,1,8,5,2,...,0,0,1,2,0,1,0,1,0,2
30488,0,0,0,0,0,0,0,0,0,0,...,1,1,1,4,6,0,1,1,1,0


# Forecast Computing

In [10]:
# Calculer la moyenne mobile sur 3 jours et arrondi haut
daily_forecast = daily_sales.rolling(window=3, axis=1).mean().apply(np.ceil)


# Renommer les colonnes de prévision
forecast_columns = [f'forecast_{col}' for col in daily_forecast.columns]
daily_forecast.columns = forecast_columns

daily_forecast


/tmp/ipykernel_6011/1203208088.py:5: FutureWarning: Support for axis=1 in DataFrame.rolling is deprecated and will be removed in a future version. Use obj.T.rolling(...) instead
  daily_forecast = daily_sales.rolling(window=3, axis=1).mean().apply(np.ceil)


,forecast_d_1,forecast_d_2,forecast_d_3,forecast_d_4,forecast_d_5,forecast_d_6,forecast_d_7,forecast_d_8,forecast_d_9,forecast_d_10,...,forecast_d_1932,forecast_d_1933,forecast_d_1934,forecast_d_1935,forecast_d_1936,forecast_d_1937,forecast_d_1938,forecast_d_1939,forecast_d_1940,forecast_d_1941
0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,2.0,2.0,2.0,0.0,0.0,1.0,2.0,2.0,2.0
1,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,1.0,2.0,2.0,1.0,1.0,0.0,0.0,0.0
2,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,1.0,1.0,1.0,0.0,1.0,2.0,2.0,2.0
3,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,3.0
4,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30485,NaN,NaN,1.0,2.0,2.0,2.0,2.0,3.0,2.0,2.0,...,1.0,1.0,2.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0
30486,NaN,NaN,0.0,0.0,0.0,2.0,2.0,2.0,1.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0
30487,NaN,NaN,2.0,3.0,2.0,3.0,3.0,5.0,5.0,5.0,...,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
30488,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.0,1.0,1.0,2.0,4.0,4.0,3.0,1.0,1.0,1.0


In [19]:
# Calculer les statistiques de forecast par produit
daily_forecast['forecast_mean'] = daily_forecast.mean(axis=1)
daily_forecast['forecast_mean_no_zeros'] = daily_forecast.replace(0, np.nan).mean(axis=1)
daily_forecast['forecast_max'] = daily_forecast.max(axis=1)
daily_forecast['forecast_std'] = daily_forecast.std(axis=1)

daily_forecast

,forecast_d_1,forecast_d_2,forecast_d_3,forecast_d_4,forecast_d_5,forecast_d_6,forecast_d_7,forecast_d_8,forecast_d_9,forecast_d_10,...,forecast_d_1936,forecast_d_1937,forecast_d_1938,forecast_d_1939,forecast_d_1940,forecast_d_1941,forecast_mean,forecast_mean_no_zeros,forecast_max,forecast_std
0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,2.0,2.0,2.0,0.494069,1.204138,3.0,0.656070
1,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.0,1.0,1.0,0.0,0.0,0.0,0.512120,1.064858,3.0,0.571050
2,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,1.0,2.0,2.0,2.0,0.271790,1.131485,3.0,0.522518
3,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.0,2.0,2.0,2.0,2.0,3.0,2.027849,2.263537,7.0,1.300192
4,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.258381,1.611392,8.0,0.995378
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30485,NaN,NaN,1.0,2.0,2.0,2.0,2.0,3.0,2.0,2.0,...,2.0,1.0,1.0,1.0,1.0,1.0,0.715317,1.530006,8.0,1.084895
30486,NaN,NaN,0.0,0.0,0.0,2.0,2.0,2.0,1.0,2.0,...,0.0,0.0,1.0,1.0,1.0,1.0,0.583290,1.219379,5.0,0.713886
30487,NaN,NaN,2.0,3.0,2.0,3.0,3.0,5.0,5.0,5.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.126870,1.689433,13.0,1.195475
30488,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,4.0,4.0,3.0,1.0,1.0,1.0,0.496132,1.636898,7.0,0.898878


# Inventory

# Up-to-level

In [ ]:
# (R, S) = (7, 3)


# Safety stock

# Sales

# Performance metrics